In [2]:
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel


def average_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]


# Each input text should start with "query: " or "passage: ", even for non-English texts.
# For tasks other than retrieval, you can simply use the "query: " prefix.
input_texts = ['query: how much protein should a female eat',
               'query: 南瓜的家常做法',
               "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 is 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or training for a marathon. Check out the chart below to see how much protein you should be eating each day.",
               "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀 6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"]

tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Tokenize the input texts
batch_dict = tokenizer(input_texts, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)
scores = (embeddings[:2] @ embeddings[2:].T) * 100
print(scores.tolist())

[[90.81393432617188, 72.13390350341797], [70.53540802001953, 88.76107788085938]]


In [5]:
import pandas as pd

df = pd.read_csv("../data/repa_data_20231022.csv")

In [7]:
embedding_list = []
for affi in df["affiliation"]:
    sentence = ['query: ' + str(affi)]
    # Tokenize the input texts
    batch_dict = tokenizer(sentence, max_length=512, padding=True, truncation=True, return_tensors='pt')

    outputs = model(**batch_dict)
    embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

    # normalize embeddings
    embedding = F.normalize(embeddings, p=2, dim=1)
    
    embedding_list.append(embedding[0])
len(scores)

In [3]:
sentence = ['query: ' + "hello world"]
# Tokenize the input texts
batch_dict = tokenizer(sentence, max_length=512, padding=True, truncation=True, return_tensors='pt')

outputs = model(**batch_dict)
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

# normalize embeddings
embeddings = F.normalize(embeddings, p=2, dim=1)

tensor([], size=(1, 0), grad_fn=<MulBackward0>)


In [4]:
print(embeddings)

tensor([[ 0.0204, -0.0168,  0.0064,  ...,  0.0033, -0.0167,  0.0212]],
       grad_fn=<DivBackward0>)


In [5]:
from sklearn.decomposition import PCA

In [8]:
embeddings.tolist()

[[0.020389847457408905,
  -0.01676425337791443,
  0.006372421979904175,
  -0.043372683227062225,
  0.04428029805421829,
  -0.0379989929497242,
  -0.039027292281389236,
  0.03480307385325432,
  0.051190610975027084,
  -0.03089717961847782,
  0.014394971542060375,
  0.032165806740522385,
  -0.029797933995723724,
  -0.02912810817360878,
  -0.02324743941426277,
  -0.009922049939632416,
  0.0018279904033988714,
  0.0025430377572774887,
  -0.007727316580712795,
  -0.0067837536334991455,
  0.003463809611275792,
  -0.041070278733968735,
  -0.018690992146730423,
  -0.06134014204144478,
  -0.016996480524539948,
  -0.0032934360206127167,
  -0.04641026258468628,
  -0.027298524975776672,
  -0.010184445418417454,
  -0.042786963284015656,
  0.000993469962850213,
  0.031012898311018944,
  -0.022952120751142502,
  -0.04292783513665199,
  -0.010273165069520473,
  0.04445286840200424,
  0.014673879370093346,
  0.02564815990626812,
  -0.05351816862821579,
  0.03156767040491104,
  -0.031736161559820175,
  

In [9]:
pca = PCA(n_components=2)
decom_embdedding = pca.fit(embeddings.tolist()).transform(embeddings.tolist())

ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'